In [ ]:
import numpy as np

import pandas as pd
from sklearn.preprocessing import LabelEncoder 
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler

train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
def encode(train, test):
    label_encoder = LabelEncoder().fit(train.species)
    labels = label_encoder.transform(train.species)
    classes = list(label_encoder.classes_)
    test_ids = test.id
    
    train = train.drop(['species', 'id'], axis=1)
    test = test.drop('id', axis=1)

    return train, labels, test, test_ids, classes

train, labels, test, test_ids, classes = encode(train, test)

# standardize train features
scaler = StandardScaler().fit(train.values)
scaled_train = scaler.transform(train.values)

# split train data into train and validation
sss = StratifiedShuffleSplit(test_size=0.1, random_state=23)
for train_index, valid_index in sss.split(scaled_train, labels):
    X_train, X_valid = scaled_train[train_index], scaled_train[valid_index]
    y_train, y_valid = labels[train_index], labels[valid_index]
    

nb_features = 64 # number of features per features type (shape, texture, margin)   
nb_class = len(classes) 

# reshape train data
X_train_r = np.zeros((len(X_train), nb_features, 3))
X_train_r[:, :, 0] = X_train[:, :nb_features]
X_train_r[:, :, 1] = X_train[:, nb_features:128]
X_train_r[:, :, 2] = X_train[:, 128:]

# reshape validation data
X_valid_r = np.zeros((len(X_valid), nb_features, 3))
X_valid_r[:, :, 0] = X_valid[:, :nb_features]
X_valid_r[:, :, 1] = X_valid[:, nb_features:128]
X_valid_r[:, :, 2] = X_valid[:, 128:]
 

 

In [ ]:
X_train_r = np.reshape(X_train_r,(891,3,8,8))
X_valid_r = np.reshape(X_valid_r,(99,3,8,8))
y_train = y_train.astype(np.int32)
 

In [ ]:
import theano
import lasagne
from lasagne import layers
from lasagne.updates import nesterov_momentum
from nolearn.lasagne import NeuralNet
from nolearn.lasagne import visualize


net1 = NeuralNet(
    layers=[('input', layers.InputLayer),
            ('conv2d1', layers.Conv2DLayer),
            ('dropout1', layers.DropoutLayer),
            ('dense1', layers.DenseLayer), 
            ('dropout2', layers.DropoutLayer),
            ('dense2', layers.DenseLayer),
            ('output', layers.DenseLayer),
            ],
    # input layer
    input_shape=(None, 3, 8, 8),
    # layer conv2d1
    conv2d1_num_filters=512,
    conv2d1_filter_size=(1, 1),
    conv2d1_nonlinearity=lasagne.nonlinearities.rectify,
    conv2d1_W=lasagne.init.GlorotUniform(),  
    # dropout1
    dropout1_p=0.5,    
    # dense1
    dense1_num_units=2048,
    dense1_nonlinearity=lasagne.nonlinearities.rectify, 
    # dropout2
    dropout2_p=0.5,  
    # dense2
    dense2_num_units=1024,
    dense2_nonlinearity=lasagne.nonlinearities.rectify,   
    # output
    output_nonlinearity=lasagne.nonlinearities.softmax,
    output_num_units=nb_class,
    # optimization method params
    update=nesterov_momentum,
    update_learning_rate=0.01,
    update_momentum=0.9,
    max_epochs=20,
    verbose=1,
    )

In [ ]:
nn = net1.fit(X_train_r, y_train)

In [ ]:
from sklearn.metrics import accuracy_score, log_loss
 
test = np.array(test)

#reshape test data
X_test = np.zeros((len(test), nb_features, 3))
X_test[:, :, 0] = test[:, :nb_features]
X_test[:, :, 1] = test[:, nb_features:128]
X_test[:, :, 2] = test[:, 128:]
X_test = np.reshape(X_test,(594,3,8,8))
 
test_predictions = net1.predict_proba(X_test)
 


In [ ]:
submission1 = pd.DataFrame(test_predictions, columns=classes)
submission1.insert(0, 'id', test_ids)
submission1.reset_index()
#submission1 = submission1.drop(submission1.columns[0], axis=1)
submission1.to_csv('submission1.csv')
submission1.tail()